In [16]:
import pandas as pd

QuestionNumber = 5  # question per page
false = False

## Functions

In [17]:
def arrange_answers(answers):    # organize answers in one instance
    arranged_answers =[[] for _ in range(QuestionNumber)]
    comment = ""
    for name in answers.keys():
        if "less-2-checkbox" in name:
            continue
        answer = answers[name].strip().lower()    # remove sapce & convert to lower
        for i in range(QuestionNumber):
            if "answer"+str(i+1)+"-" in name:     # belong to question i
                arranged_answers[i].append(answer)
            if "in-answer-box"+str(i+1) in name:
                arranged_answers[i].append(answer)
            if name == "comments":
                comment = answer
    return [sorted(set(t), key=t.index) for t in  arranged_answers],comment # remove repeated answer # keep the answer order



def answer_frequency(Answers, Total_number):          # count the frequency of each answer
    ritem = dict()
    for ans in Answers:
        if ans in ritem: ritem[ans][0] += 1
        else: ritem[ans] = [1,Total_number]
    return ritem

## Collect information & answers & review result

In [18]:
original_result = pd.read_csv("Batch_4282865_batch_results.csv")

In [19]:
## HIT-information dictionary
hit_info = dict()

# scan batch result
last_hit_id = None
for ri, (index, row) in enumerate(original_result.iterrows()):
    current_hit_id = row["HITId"]
    if (last_hit_id is not None) and (current_hit_id == last_hit_id):                              # same HIT
        line_answer,comment = arrange_answers(eval(row["Answer.taskAnswers"])[0])
        for i in range(QuestionNumber):
            answers[i].append(line_answer[i])
    if (last_hit_id is not None) and (current_hit_id!=last_hit_id or ri==len(original_result)-1):  # a new HIT or the Last row
        hit_info[last_hit_id] = {"VideoIds":video_ids,"Questions":video_questions,"StartTimes":start_times,
                                 "EndTimes":end_times,"StdAnswers":std_answers,"Answers":[]}
        for fwid in range(QuestionNumber):
            qanswers = []                                      # answers for one question
            for canid in range(len(answers[fwid])):
                qanswers.append(answers[fwid][canid])
            hit_info[last_hit_id]["Answers"].append(qanswers)
    if  last_hit_id == None or current_hit_id!=last_hit_id:    # collect information & answers for the new HIT
        last_hit_id = current_hit_id
        video_ids = [row["Input.video"+str(wid)+"_id"] for wid in range(1,QuestionNumber+1)]
        video_questions = [row["Input.question"+str(wid)] for wid in range(1,QuestionNumber+1)]
        start_times = [row["Input.video"+str(wid)+"_start_time"] for wid in range(1,QuestionNumber+1)]
        end_times = [row["Input.video"+str(wid)+"_end_time"] for wid in range(1,QuestionNumber+1)]
        std_answers = [row["Input.label"+str(wid+1)] for wid in range(QuestionNumber)]
        answers = [[] for _ in range(QuestionNumber)]          # answers for one HIT
        line_answer,comment = arrange_answers(eval(row["Answer.taskAnswers"])[0])
        for i in range(QuestionNumber):
            answers[i].append(line_answer[i])

In [20]:
# HIT-review result dictionary
hit_review = dict()

with open("Review_result_instance.txt","r") as f:
    for line in f.readlines():
        if("HITId:" in line):
            current_hit = line[7:-1]
            hit_review[current_hit] = []
        elif("Appr "in line):
            hit_review[current_hit].append(True)
        elif("Reje " in line or "Midd " in line):
            hit_review[current_hit].append(False)

## Conbine approved answers & write into file

In [21]:
titles = ["video_id","question","video_start_time","video_end_time","standard_answer","worker_answers"]
records = []
for hit in hit_info.keys():
    review_result = hit_review[hit]
    info_answers = hit_info[hit]
    for qind in range(QuestionNumber):
        record = [info_answers["VideoIds"][qind], info_answers["Questions"][qind],info_answers["StartTimes"][qind],
                 info_answers["EndTimes"][qind],info_answers["StdAnswers"][qind]]
        approved_answers = [info_answers["Answers"][qind][idx] for idx in range(len(review_result)) if review_result[idx]]
        not_empty_answers = sum([ aitem != [] for aitem in approved_answers])    # Amount of not empty answers
        all_appr_answers = [ite2 for ite1 in approved_answers for ite2 in ite1]
        if sum(["unavailable video" == ite for ite in all_appr_answers]) >= 2:   # unavailable video
            record.append([])
        else:
            all_appr_freq = answer_frequency(all_appr_answers,not_empty_answers) # Frequency of each anser
            if "unavailable video" in all_appr_freq:                             # False "unvavilable video" due to worker's reason
                all_appr_freq.pop("unavailable video")
            record.append(all_appr_freq)
        records.append(record)

        df = pd.DataFrame(records,columns=titles)

In [22]:
df.to_csv("organized_approve_answers.csv",index=False)